# Handling filter objects

The most fundamental building block of photometric surveys lies in the bandpass filters used to conduct them. In this example we will learn how to use the `Filter` class which is used in (nearly) every use-case of the `galfind` code. We start by looking at the JWST/NIRCam/F444W band, which is very commonly used in both blank field and cluster surveys.

In [ ]:
# imports
from galfind import Filter, Multiple_Filter

# Example 1: Create a filter object from a filter name
facility = "JWST"
instrument = "NIRCam"
filter_name = "F444W"
filt = Filter.from_SVO(facility, instrument, filter_name)